In [1]:
##Idea is to get all relavant departments because the API had a ton of noise that would make the program much slower
##and much more difficult to debug. I got the relavant departments by transcribing the availible ones on Dukehub.

##ME, OVS excluded because not on database, I do not know why ME is not because it is very common

courses_we_want = ["AAAS","ACCOUNTG","AEROSCI","AMES","AMI","AMXTIAN","ANESTH","ARABIC","ARTHIST","ARTS&SCI","ARTSVIS","ASEP"]
courses_we_want.extend(["BA","BCS","BES","BIOCHEM","BIOETHIC","BIOLOGY","BIOSTAT","BME","BRAINSOC","BSP"])
courses_we_want.extend(["CBB","CEE","CELLBIO","CESC","CHEM","CHILDPOL","CHINESE","CHURHST","CHURMIN","CLP","CLST","CMAC","CMB","COMMFAM","COMPSCI","CONTDIV","CONTENV","CONTNUR","CONTPPS","CPE","CROELE","CRP","CRS","CRSP","CTN","CULANTH","CVS"])
courses_we_want.extend(["DANCE","DECISION","DESCI","DMNISTRY","DOCST"])
courses_we_want.extend(["ECE","ECON","EDUC","EGR","EGRMGMT","EHD","ENERGY","ENGLISH","ENNGYEGR","ENRGYENV","ENVIRON","EOS","EPH","ETHICS","EVANTH"])
courses_we_want.extend(["FIELDEDU","FINANCE","FREETIME","FRENCH","FUQINTRD"])
courses_we_want.extend(["GATE","GENOME","GERMAN","GHS","GLHLTH","GREEK","GS","GSF"])
courses_we_want.extend(["HCVIS","HEBREW","HGP","HINDI","HISTORY","HISTREL","HISTTHEO","HLTHMGMT","HLTHSCI","HOUSECS","HUMANDEV"])
courses_we_want.extend(["I&E","ICS","IDS","IMMUNOL","INTERDIS","ISS","ITALIAN"])
courses_we_want.extend(["JEWISHST","JPN"])
courses_we_want.extend(["KOREAN"])
courses_we_want.extend(["LATAMER","LATIN","LAW","LINGUIST","LIT","LS","LSGS","LTS"])
courses_we_want.extend(["MANAGEMT","MARKETING","MAT","MATH","MEDHUM","MEDICINE","MEDPHY","MEDREN","MENG","MFAEDA","MGM","MGMTCOM","MGRECON","MIDIP","MILITSCI","MMCI","MMS","MOLCAN","MOLMED","MSIS","MSLS","MUSIC"])
courses_we_want.extend(["NAONOSCI","NAVALSCI","NCS","NEURO","NEUROBIO","NEUROSCI","NEUROSUR","NEWTEST","NSS","NURSING"])
courses_we_want.extend(["OBGYN","OLDTEST","OPERATNS","OPHTHAL","OPTECH","OPTRS","ORTHO"])
courses_we_want.extend(["PARISH","PASTCARE","PATHASST","PATHOL","PCLT","PEDS","PERSIAN","PHARM","PHIL","PHYASST","PHYSEDU","PHYSICS","PJMS","PMT","POLISH","POLSCI","PORTUGUE","PREACHING","PSP","PSY","PSYCHTRY","PT-D","PUBPOL"])
##none that start with Q
courses_we_want.extend(["RADIOL","RADONC","REG","RELIGION","RESEARCH","RIGHTS","ROMST","RROMP","RUSSIAN"])
courses_we_want.extend(["SBB","SCISOC","SES","SOCIOL","SPANISH","SPIRIT","STA","STDYAWAY","STRATEGY","SURGERY","SXL"])
courses_we_want.extend(["THEATRST","THESIS","TIBETAN","TURKISH"])
courses_we_want.extend(["UPE","UPGEN"])
courses_we_want.extend(["VMS"])
courses_we_want.extend(["WRITING","WXTIAN"])
courses_we_want.extend(["XTIANEDU","XTIANETH","XTIANPRC","XTIANTHE"])
##none that start with Y
##none that start with Z

In [2]:
import requests as rq
def get_all_subjects(): ##Get course departments and longer name for all departments from above. Used to be more useful
                        ##When did not have the above courses_we_want table. Was also full of edge case if statements
    all_courses = []
    ##found URL based on Duke API. 
    data = rq.get("https://streamer.oit.duke.edu/curriculum/list_of_values/fieldname/SUBJECT?access_token=f3849db401a3437f92241b5da3b98e1b").json()
    ##found path based on API as well
    classes = data["scc_lov_resp"]["lovs"]["lov"]["values"]["value"]
    for course in classes:
        if str(course["code"]) not in courses_we_want: ##It is a weird department
            continue
        all_courses.append([str(course["code"]),str(course["desc"])])
    return all_courses
complete_course_list = get_all_subjects()
# print complete_course_list

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [26]:
# print complete_course_list[4]
token = "f3849db401a3437f92241b5da3b98e1b" ##my API token I got for this assignment
base_url = "https://streamer.oit.duke.edu/curriculum/courses/subject/"

def get_ids_and_numbers(Department): ##gets class number and also other portion to make it unique
    answer = []
    add_on = (Department[0] + " - " + Department[1]).replace(" ","%20") + "?" + "access_token=" + token
#     print base_url + add_on
    data = rq.get(base_url + add_on).json()
#print_json_tree(data["ssr_get_courses_resp"]["course_search_result"]["subjects"]["subject"])
#     print Department[0]
    if data["ssr_get_courses_resp"]["course_search_result"]["ssr_crs_gen_msg"] == "The Search has returned no results":
        return "skip" ##I return skip if the course cannot be found in the API
    if "ssr_get_courses_resp" not in data.keys():
        return "skip" ##This is an edge case for one bad data point
    courses = data["ssr_get_courses_resp"]["course_search_result"]["subjects"]["subject"]["course_summaries"]["course_summary"]
    ids_and_numbers = []
#     print len(courses)
    if type(courses) == list: ##it would be a dict otherwise and could be added to the list straight away
        for i in range(len(courses)): ##add all courses from that department to the list of courses
            course = courses[i]
            if "msg_txt" in course.keys():
                if course["msg_txt"] == "** available as of 2019-01-01":
                    continue ##This class has not started yet and nobody could thus be in this class ever
            to_append = [str(course["crse_id"]),str(course["crse_offer_nbr"])]
            answer.append(to_append)
    else:
        to_append = [str(courses["crse_id"]),str(courses["crse_offer_nbr"])]
        answer.append(to_append)

    return answer
ids_and_numbers = get_ids_and_numbers(complete_course_list[4])
# print ids_and_numbers


[['018029', '1'], ['019765', '1'], ['001873', '4'], ['024999', '2'], ['024426', '2'], ['021768', '1'], ['004558', '1'], ['021769', '1'], ['022108', '3'], ['023563', '1'], ['022989', '1'], ['003845', '3'], ['018102', '1'], ['025329', '1'], ['006321', '2'], ['019781', '2'], ['017078', '2'], ['022160', '2'], ['018715', '3'], ['021858', '1'], ['024152', '1'], ['021859', '1'], ['004565', '1'], ['012901', '1'], ['020269', '2'], ['017077', '6'], ['020222', '3'], ['020830', '1'], ['019110', '6'], ['019346', '7'], ['022593', '3'], ['024709', '3'], ['006322', '3'], ['023326', '3'], ['023920', '2'], ['023910', '2'], ['014341', '2'], ['023632', '2'], ['022997', '3'], ['020610', '2'], ['021369', '2'], ['021331', '2'], ['006320', '5'], ['013901', '2'], ['000040', '3'], ['024130', '2'], ['019451', '3'], ['019049', '2'], ['000028', '3'], ['016582', '3'], ['000029', '3'], ['016410', '3'], ['018481', '2'], ['016082', '2'], ['004492', '2'], ['016992', '3'], ['020119', '2'], ['022308', '2'], ['005278', '3

In [27]:
attribute_list = ["(ALP)","(CCI)","(CZ)","(EI)","(FL)","(NS)","(QS)","(SS)","(R)","(SEM)","(STS)","(W)"]
##the attribute list has () around the attributes because that is how they appear in the api
terms_offered = ["2012 Fall Term", "2013 Spring Term","2013 Fall Term","2014 Spring Term","2014 Fall Term","2015 Spring Term","2015 Fall Term","2016 Spring Term","2016 Fall Term","2017 Spring Term","2017 Fall Term","2018 Spring Term","2018 Fall Term","2019 Spring Term"]
##I chose to only focus on the spring and fall terms becaue I felt that was easiest. The API goes back to 2012 fall at 
##its earliest
def create_terms(terms_offered, start_number): ##method that creates terms to be the way the API has it. This was
    answer = []                                ##slightly more complicated than I thought it would be
    number = start_number - 30
    for i in range(len(terms_offered)):
        if i%2 != 0:
            number = number + 10
            answer.append(str(number) + " - " + terms_offered[i])
        else:
            number = number + 30
            answer.append(str(number) + " - " + terms_offered[i])
    return answer

all_terms = create_terms(terms_offered,1420) ##storing all terms in all_terms variable       

In [150]:
url_section0 = "https://streamer.oit.duke.edu/curriculum/classes/strm/"
# 1670%20-%202019%20Spring%20Term/crse_id/000081/crse_offer_nbr/1/session_code/1/class_section/01?access_token=f3849db401a3437f92241b5da3b98e1b
def make_course(course_id,course_offer_number,time_period):
    course = []
    url_section1 = time_period.replace(" ","%20")
    url_section2 = "/crse_id/"
    url_section3 = course_id
    url_section4 = "/crse_offer_nbr/"
    url_section5 = course_offer_number
    url_section6 = "/session_code/1/class_section/01?access_token="
    url_full = url_section0 + url_section1 + url_section2 + url_section3 + url_section4 + url_section5 + url_section6 + token
#     print url_full
    data = rq.get(url_full).json()
#   if data["ssr_get_course_offering_resp"]["course_offering_result"]["ssr_crs_gen_msg"] == "***  This course has not been scheduled.  ***":
#         url = url_base
# #             print "exiting because course never offered"
#         continue
#     if "course_offering" not in data["ssr_get_course_offering_resp"]["course_offering_result"]:
#         url = url_base
# #             print "exiting because course offering never happened"
#         continue
    if "ssr_get_class_section_resp" not in data.keys(): ##edge case
        return []
    if data["ssr_get_class_section_resp"]["class_section_result"]["class_sections"] == data["ssr_get_class_section_resp"]["class_section_result"]["ssr_crs_gen_msg"]: 
        return [] ##edge case where nothing was there. This was the way I found to capture it but theres probs a better way
        
    class_section = data["ssr_get_class_section_resp"]["class_section_result"]["class_sections"]["ssr_class_section"]
    
    
    name = str(class_section["crse_id_lov_descr"])
    subject = str(class_section["subject"])
    course_number = str(class_section["catalog_nbr"])[1:]
    prereqs = str(class_section["ssr_requisite_long"])
    
        
    attributes = []
    if class_section["ssr_component"] == "SEM": ##I chose to abbreviate seminar as S not SEM
        attributes.append("S")
    
    class_meeting = class_section["class_meeting_patterns"]["class_meeting_pattern"]
    if len(class_meeting) < 5:
        class_meeting = class_meeting[0]
    meeting_times = str(class_meeting["ssr_mtg_sched_long"])
    meeting_location = str(class_meeting["ssr_mtg_loc_long"])
    instructor = str(class_meeting["ssr_instr_long"])
    

    for attribute in attribute_list:
        if class_section["enrollment_details"]["enrollment_information"]["class_attributes"]:
            if attribute in class_section["enrollment_details"]["enrollment_information"]["class_attributes"]:
                attributes.append(attribute[1:-1])
    
                                                                
    return [course_id,name,subject,course_number,time_period[7:],prereqs,meeting_times,meeting_location,instructor,attributes]
    ## I returned this because this is all the useful data that I would need from this specific API category. This 
    ## is the most important returned value
# print make_course(ids_and_numbers[0][0],ids_and_numbers[0][1],all_terms[0])


In [166]:
ids_and_numbers = []
for i in range((len(complete_course_list))):
    course_list = get_ids_and_numbers(complete_course_list[i])
    ids_and_numbers.extend(course_list)
    print len(ids_and_numbers)

191
230
239
486
687
708
718
743
977
1004
1117
1118
1150
1169
1171
1198
1219
1442
1487
1600
1605
1606
1630
1737
1754
1756
1827
1838
1876
1922
1931
1947
2055
2078
2097
2115
2205
2207
2211
2215
2216
2235
2264
2265
2266
2270
2516
2517
2616
2655
2662
2805
2929
3216
3296
3330
3350
3369
3412
3644
3656
4070
4168
4169
4257
4341
4346
4396
4397
4508
4546
4556
4566
4677
4678
4869
4906
4943
5094
5103
5115
5116
5128
5577
5587
5611
5638
5664
5665
5671
5718
5998
6019
6031
6065
6190
6284
6350
6366
6377
6410
6457
6857
6939
7201
7205
7230
7249
7307
7315
7484
7485
7530
7595
7736
7740
7750
7777
7801
7829
7830
7847
7864
7887
7900
7905
7906
7907
8106
8121
8123
8131
8162
8270
8274
8358
8359
8596
8612
8689
8716
8720
8746
8749
8762
8825
8846
8872
8889
8890
8932
8940
8975
9119
9215
9304
9408
9451
9452
9469
9903
9934
9935
10161
10172
10217
10624
10634
10636
10648
10969
10970
11105
11185
11186
11356
11362
11411
11468
11622
11742
11747
11833
11842
11887
11926
11951
12077
12078
12088
12107
12112
12142
12465
12483
12

In [173]:
for i in range(len(ids_and_numbers)):
    print i, ids_and_numbers[i]

0 ['000064', '1']
1 ['000081', '1']
2 ['000082', '1']
3 ['022918', '2']
4 ['019142', '4']
5 ['017879', '2']
6 ['020070', '3']
7 ['020130', '2']
8 ['018580', '2']
9 ['007033', '1']
10 ['025035', '2']
11 ['025250', '2']
12 ['025328', '2']
13 ['024456', '2']
14 ['000074', '1']
15 ['000067', '1']
16 ['018192', '1']
17 ['000075', '1']
18 ['025265', '2']
19 ['005262', '1']
20 ['005263', '1']
21 ['005338', '1']
22 ['005186', '1']
23 ['024695', '5']
24 ['005180', '1']
25 ['022535', '2']
26 ['019171', '4']
27 ['005206', '1']
28 ['005330', '1']
29 ['005211', '1']
30 ['005212', '1']
31 ['012954', '3']
32 ['022920', '3']
33 ['016405', '5']
34 ['003930', '1']
35 ['003931', '1']
36 ['020222', '2']
37 ['021134', '2']
38 ['021786', '2']
39 ['000610', '1']
40 ['023652', '2']
41 ['021055', '2']
42 ['017885', '2']
43 ['017407', '2']
44 ['024711', '2']
45 ['024716', '2']
46 ['020146', '3']
47 ['021059', '2']
48 ['021822', '2']
49 ['022121', '2']
50 ['014812', '3']
51 ['005319', '1']
52 ['020131', '2']
53 

677 ['019782', '1']
678 ['020671', '1']
679 ['025661', '1']
680 ['025662', '1']
681 ['025663', '1']
682 ['023556', '2']
683 ['025555', '3']
684 ['021181', '1']
685 ['025282', '2']
686 ['025076', '1']
687 ['000208', '1']
688 ['020185', '1']
689 ['018477', '1']
690 ['000210', '1']
691 ['020527', '1']
692 ['021103', '1']
693 ['000213', '1']
694 ['006490', '1']
695 ['000215', '1']
696 ['000219', '1']
697 ['018740', '1']
698 ['000222', '1']
699 ['000217', '1']
700 ['022502', '1']
701 ['022917', '1']
702 ['023193', '1']
703 ['024975', '1']
704 ['016995', '1']
705 ['000223', '1']
706 ['000224', '1']
707 ['018540', '1']
708 ['016306', '1']
709 ['016976', '1']
710 ['023836', '1']
711 ['023837', '1']
712 ['025499', '1']
713 ['017060', '1']
714 ['010571', '1']
715 ['010572', '1']
716 ['014607', '1']
717 ['016015', '1']
718 ['000506', '1']
719 ['000507', '1']
720 ['000512', '1']
721 ['000513', '1']
722 ['021100', '1']
723 ['000527', '1']
724 ['021898', '1']
725 ['022530', '1']
726 ['023314', '1']


1282 ['001520', '1']
1283 ['020062', '4']
1284 ['023322', '1']
1285 ['001475', '1']
1286 ['017256', '1']
1287 ['001547', '1']
1288 ['023990', '1']
1289 ['001543', '1']
1290 ['001542', '1']
1291 ['021386', '1']
1292 ['021475', '1']
1293 ['021047', '1']
1294 ['021772', '2']
1295 ['018500', '1']
1296 ['023906', '2']
1297 ['015730', '1']
1298 ['015730', '1']
1299 ['020458', '2']
1300 ['021050', '1']
1301 ['025322', '1']
1302 ['017112', '1']
1303 ['017112', '1']
1304 ['001521', '1']
1305 ['001482', '1']
1306 ['024645', '2']
1307 ['024178', '2']
1308 ['016051', '1']
1309 ['001514', '1']
1310 ['020059', '1']
1311 ['019369', '2']
1312 ['024350', '1']
1313 ['025033', '1']
1314 ['025003', '1']
1315 ['015486', '1']
1316 ['015486', '1']
1317 ['019728', '1']
1318 ['025137', '2']
1319 ['019725', '1']
1320 ['025216', '1']
1321 ['001489', '1']
1322 ['013593', '1']
1323 ['016480', '1']
1324 ['001499', '1']
1325 ['001500', '1']
1326 ['001501', '1']
1327 ['001535', '1']
1328 ['020471', '2']
1329 ['022873

1920 ['023864', '1']
1921 ['018478', '1']
1922 ['002523', '1']
1923 ['002524', '1']
1924 ['017806', '1']
1925 ['018808', '1']
1926 ['016850', '1']
1927 ['002528', '1']
1928 ['014513', '1']
1929 ['015558', '1']
1930 ['023189', '1']
1931 ['010729', '1']
1932 ['014441', '1']
1933 ['014514', '1']
1934 ['014538', '1']
1935 ['014539', '1']
1936 ['014440', '1']
1937 ['014536', '1']
1938 ['014537', '1']
1939 ['015920', '1']
1940 ['016332', '1']
1941 ['019937', '1']
1942 ['023035', '1']
1943 ['023036', '1']
1944 ['023037', '1']
1945 ['023038', '1']
1946 ['024740', '1']
1947 ['002728', '1']
1948 ['006622', '2']
1949 ['022462', '1']
1950 ['002759', '1']
1951 ['024705', '1']
1952 ['023434', '1']
1953 ['023828', '1']
1954 ['002723', '1']
1955 ['002726', '1']
1956 ['018230', '2']
1957 ['002746', '1']
1958 ['021303', '1']
1959 ['023292', '1']
1960 ['022830', '6']
1961 ['023827', '1']
1962 ['000546', '2']
1963 ['000546', '2']
1964 ['023216', '1']
1965 ['015846', '2']
1966 ['000563', '2']
1967 ['022171

2406 ['019776', '2']
2407 ['019872', '3']
2408 ['020117', '2']
2409 ['022532', '2']
2410 ['025261', '4']
2411 ['014666', '4']
2412 ['023934', '4']
2413 ['022235', '2']
2414 ['009654', '3']
2415 ['025246', '2']
2416 ['023347', '1']
2417 ['020254', '1']
2418 ['020240', '1']
2419 ['020270', '1']
2420 ['021084', '2']
2421 ['020278', '2']
2422 ['023884', '3']
2423 ['020049', '1']
2424 ['024244', '2']
2425 ['022361', '2']
2426 ['022582', '3']
2427 ['001955', '1']
2428 ['013132', '1']
2429 ['015488', '1']
2430 ['015489', '1']
2431 ['017935', '1']
2432 ['018531', '1']
2433 ['016053', '1']
2434 ['019777', '1']
2435 ['019779', '1']
2436 ['015192', '1']
2437 ['021347', '1']
2438 ['020755', '1']
2439 ['022461', '1']
2440 ['021120', '1']
2441 ['024757', '3']
2442 ['022581', '2']
2443 ['024157', '5']
2444 ['024698', '2']
2445 ['022871', '2']
2446 ['021345', '2']
2447 ['025138', '1']
2448 ['017073', '3']
2449 ['024120', '2']
2450 ['021753', '1']
2451 ['020494', '2']
2452 ['001960', '1']
2453 ['001961

...

9691 ['021545', '1']
9692 ['019032', '1']
9693 ['019033', '1']
9694 ['019034', '1']
9695 ['019035', '1']
9696 ['021392', '1']
9697 ['021478', '1']
9698 ['019028', '1']
9699 ['019029', '1']
9700 ['019030', '1']
9701 ['019031', '1']
9702 ['021480', '1']
9703 ['021520', '1']
9704 ['021095', '4']
9705 ['020213', '3']
9706 ['023914', '3']
9707 ['022170', '1']
9708 ['020469', '1']
9709 ['024697', '3']
9710 ['024157', '3']
9711 ['020493', '2']
9712 ['008459', '1']
9713 ['025595', '2']
9714 ['022135', '1']
9715 ['002804', '2']
9716 ['019037', '1']
9717 ['019038', '1']
9718 ['019039', '1']
9719 ['019040', '1']
9720 ['021548', '1']
9721 ['021538', '1']
9722 ['021887', '2']
9723 ['023877', '1']
9724 ['021855', '3']
9725 ['008521', '1']
9726 ['021511', '1']
9727 ['008524', '1']
9728 ['018874', '2']
9729 ['008559', '1']
9730 ['008532', '1']
9731 ['017406', '1']
9732 ['008529', '1']
9733 ['015212', '1']
9734 ['008528', '1']
9735 ['005394', '2']
9736 ['019716', '1']
9737 ['008538', '1']
9738 ['008189

10344 ['023570', '1']
10345 ['024054', '2']
10346 ['006412', '4']
10347 ['020131', '1']
10348 ['025146', '1']
10349 ['025030', '1']
10350 ['008068', '2']
10351 ['008071', '1']
10352 ['015570', '1']
10353 ['020501', '1']
10354 ['024695', '3']
10355 ['003292', '2']
10356 ['023225', '2']
10357 ['023281', '4']
10358 ['000337', '4']
10359 ['025051', '2']
10360 ['021418', '3']
10361 ['025023', '5']
10362 ['023873', '2']
10363 ['023887', '5']
10364 ['023888', '3']
10365 ['017291', '1']
10366 ['017949', '2']
10367 ['003313', '2']
10368 ['023220', '2']
10369 ['024197', '1']
10370 ['021436', '3']
10371 ['023949', '2']
10372 ['021095', '5']
10373 ['024416', '2']
10374 ['024468', '3']
10375 ['025333', '2']
10376 ['020618', '2']
10377 ['022580', '2']
10378 ['023868', '2']
10379 ['024194', '3']
10380 ['022844', '2']
10381 ['024257', '3']
10382 ['024418', '5']
10383 ['024905', '3']
10384 ['023914', '2']
10385 ['022831', '2']
10386 ['008458', '2']
10387 ['024199', '3']
10388 ['024652', '4']
10389 ['02

10939 ['016025', '1']
10940 ['022625', '1']
10941 ['013805', '1']
10942 ['021471', '1']
10943 ['009496', '1']
10944 ['021472', '1']
10945 ['009498', '1']
10946 ['009463', '1']
10947 ['015804', '2']
10948 ['023848', '1']
10949 ['015651', '1']
10950 ['024596', '1']
10951 ['024101', '1']
10952 ['025524', '1']
10953 ['016476', '1']
10954 ['009492', '1']
10955 ['009493', '1']
10956 ['014645', '1']
10957 ['009534', '1']
10958 ['012221', '1']
10959 ['023832', '1']
10960 ['024223', '1']
10961 ['009473', '1']
10962 ['009520', '1']
10963 ['024402', '1']
10964 ['009513', '1']
10965 ['009536', '1']
10966 ['009539', '1']
10967 ['009540', '1']
10968 ['016791', '1']
10969 ['011437', '1']
10970 ['025168', '1']
10971 ['023954', '4']
10972 ['020823', '2']
10973 ['023540', '5']
10974 ['023568', '3']
10975 ['007099', '3']
10976 ['023214', '7']
10977 ['024147', '2']
10978 ['020151', '3']
10979 ['020070', '5']
10980 ['025125', '1']
10981 ['025126', '1']
10982 ['023957', '5']
10983 ['024709', '4']
10984 ['01

11612 ['009961', '1']
11613 ['009973', '1']
11614 ['009980', '1']
11615 ['019066', '1']
11616 ['018740', '2']
11617 ['019071', '1']
11618 ['010002', '1']
11619 ['010016', '1']
11620 ['023152', '1']
11621 ['022689', '1']
11622 ['018640', '1']
11623 ['010030', '1']
11624 ['010020', '1']
11625 ['010022', '1']
11626 ['010025', '1']
11627 ['016557', '1']
11628 ['022918', '5']
11629 ['019142', '3']
11630 ['021365', '1']
11631 ['010031', '1']
11632 ['010035', '1']
11633 ['024343', '1']
11634 ['024874', '1']
11635 ['010026', '1']
11636 ['016558', '1']
11637 ['021957', '2']
11638 ['023653', '2']
11639 ['023686', '2']
11640 ['023201', '1']
11641 ['010038', '1']
11642 ['013025', '1']
11643 ['013026', '1']
11644 ['010047', '1']
11645 ['010039', '1']
11646 ['015778', '1']
11647 ['021857', '1']
11648 ['016899', '1']
11649 ['018273', '1']
11650 ['023933', '1']
11651 ['022213', '1']
11652 ['022868', '1']
11653 ['022934', '1']
11654 ['022955', '1']
11655 ['024292', '1']
11656 ['025174', '1']
11657 ['01

12156 ['018852', '3']
12157 ['022020', '3']
12158 ['019364', '1']
12159 ['021773', '1']
12160 ['020128', '1']
12161 ['022108', '1']
12162 ['023563', '3']
12163 ['011802', '5']
12164 ['017367', '1']
12165 ['017366', '2']
12166 ['016079', '3']
12167 ['023900', '4']
12168 ['013907', '1']
12169 ['013031', '3']
12170 ['000546', '3']
12171 ['000546', '3']
12172 ['015760', '3']
12173 ['024221', '3']
12174 ['017875', '3']
12175 ['021858', '4']
12176 ['000977', '3']
12177 ['016425', '3']
12178 ['017089', '3']
12179 ['016077', '3']
12180 ['000982', '2']
12181 ['022593', '2']
12182 ['017077', '5']
12183 ['018247', '4']
12184 ['009676', '3']
12185 ['023920', '3']
12186 ['023910', '3']
12187 ['017356', '3']
12188 ['000077', '2']
12189 ['000093', '3']
12190 ['015253', '2']
12191 ['000028', '4']
12192 ['000029', '4']
12193 ['009249', '2']
12194 ['016410', '4']
12195 ['000040', '4']
12196 ['015830', '1']
12197 ['019055', '3']
12198 ['019110', '5']
12199 ['019440', '3']
12200 ['002747', '3']
12201 ['00

In [231]:
all_classes = []
for j in range(341,(len(ids_and_numbers))):
    if ids_and_numbers[j] == "skip":
        continue
    for k in range(len(all_terms)):
#         print k
        answer = make_course(ids_and_numbers[j][0],ids_and_numbers[j][1],all_terms[k])
        if answer != []:
            print answer
            all_classes.append(answer)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [230]:
file_classes = open("Classes Progress.txt", "r") 
all_classes = []
for line in file_classes: 
    sections = line.split("',")
    if len(line) == 1:
        continue
    a = sections[0][2:]
    b = sections[1][2:]
    c = sections[2][2:]
    d = sections[3][2:]
    e = sections[4][2:]
    f = sections[5][2:]
    g = sections[6][2:]
    h = sections[7][2:]
    i = sections[8][2:]
    j = ",".join(sections[9:])[2:-3].replace("'","").replace(" ","").split(",")
    all_classes.append([a,b,c,d,e,f,g,h,i,j])

for course in all_classes:
    print course

print len(all_classes)

['000064', 'FIRST-YEAR SEMINAR (TOP)', 'AAAS', ' 89S', '2012 Fall Term', 'Prereq: First year or Transfer students only', 'Tu 4:40PM - 7:10PM', 'Bivins 109', 'Bayo Holsey', ['S', 'CCI', 'SS']]
['000064', 'FIRST-YEAR SEMINAR (TOP)', 'AAAS', ' 89S', '2013 Spring Term', 'Prereq: First year or Transfer students only', 'TuTh 10:05AM - 11:20AM', 'Friedl Bldg 102', 'Wahneema Lubiano', ['S', 'ALP', 'CCI', 'CZ', 'EI']]
['000064', 'FIRST-YEAR SEMINAR (TOP)', 'AAAS', ' 89S', '2013 Fall Term', 'Prereq: First year or Transfer students only', 'Tu 4:40PM - 7:10PM', 'Friedl Bldg 240', 'Bayo Holsey', ['S', 'CCI', 'SS']]
['000064', 'FIRST-YEAR SEMINAR (TOP)', 'AAAS', ' 89S', '2014 Fall Term', 'Prereq: First year or Transfer students only', 'TBA', 'TBA', 'Staff', ['S', 'CCI', 'CZ', 'SS']]
['000064', 'FIRST-YEAR SEMINAR (TOP)', 'AAAS', ' 89S', '2015 Fall Term', 'Prereq: First year, First year with exception or Transfer students only', 'Tu 3:05PM - 5:35PM', 'Allen 226', 'Angel Harris', ['S', 'CCI', 'EI', 'S

['020263', 'PIGGING OUT: CULTURAL POL/FOOD', 'AAAS', '352', '2018 Fall Term', 'None', 'W 11:45AM - 2:15PM', 'West Duke 202', 'Michaeline A Crichlow', ['CCI', 'EI', 'SS']]
['025114', 'FRANCE AND AFRICA', 'AAAS', '380', '2018 Spring Term', 'None', 'MW 11:45AM - 1:00PM', 'Languages 114', 'Stephen William Smith', ['CCI', 'CZ']]
['025114', 'FRANCE AND AFRICA', 'AAAS', '380', '2019 Spring Term', 'None', 'WF 11:45AM - 1:00PM', 'Allen 226', 'Stephen W Smith', ['CCI', 'CZ']]
['025118', 'FRANCE & AFRICA-FRENCH PRECEP', 'AAAS', '380P', '2018 Spring Term', 'None', 'W 4:45PM - 6:00PM', 'Languages 208', 'Stephen William Smith', ['']]
['025118', 'FRANCE & AFRICA-FRENCH PRECEP', 'AAAS', '380P', '2019 Spring Term', 'None', 'W 4:40PM - 6:00PM', 'Languages 208', 'Stephen W Smith', ['']]
['022235', 'RACE/CULTURE OF AFRI DIASPORA', 'AAAS', '385S', '2013 Fall Term', 'None', 'Th 4:40PM - 7:10PM', 'Friedl Bldg 240', 'James Lorand Matory', ['S', 'CCI', 'CZ', 'SS']]
['022235', 'RACE/CULTURE OF AFRI DIASPORA', '

In [175]:
ls

Classes Progress.rtf     course.csv               takes_course.csv
Compsci 316 Final.ipynb  in_study_group.csv       tas_course.csv
Major Likelyhood.csv     professor.csv            teaches_course.csv
Netids.csv               rates_course.csv         tutor.csv
Teachers Raw.csv         student.csv              tutors_for.csv
all_classes.csv          study_group.csv


In [157]:
# all_classes = []
# ##this is the section where I actually do stuff. I had to put a couple of brakes or else it would take hours to run
# for i in range(len(complete_course_list)):
#     print complete_course_list[i]
# #     print i
#     ids_and_numbers = get_ids_and_numbers(complete_course_list[i])
#     if ids_and_numbers == "skip":
#         continue
#     for j in range(len(ids_and_numbers)):
# #         if j == 5: #brake one. It only allows there to be 5 types of ids_and_numbers per department
# #             break
#         for k in range(len(all_terms)): #brake two. It only includes the most recent 3 or 4 terms I forget
#             answer = make_course(ids_and_numbers[j][0],ids_and_numbers[j][1],all_terms[k])
#             if answer != []:
#                 all_classes.append(answer)
# #                 print answer
# ##My idea is to get a basic list for the second check in and then take off the brakes when we have more time. This would
# ##take hours if not days to run if no brakes were there and enough functionality is there even with brakes

['AAAS', 'African and African American S']


KeyboardInterrupt: 

In [58]:
with open('all_classes.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for course in all_classes:
        prereqs = course[5].replace(",",":")
        name = course[1].replace(","," ")
        teacher = course[8].replace(",",":").replace("\n","")
        tags = ":".join(course[9])
        spamwriter.writerow([str(course[0]),name,course[2],course[3],course[4],prereqs,course[6],course[7], teacher, tags])
#         spamwriter.writerow([course[0],course[1],course[2],course[3],course[4],course[5],course[6],course[7],course[8]])

In [59]:
teaches_course = [] ##final table
courses = [] ##final table
for course in all_classes: ## all information can be made from 
    teaches_course.append([course[0],course[4],"1",course[-2],course[2]])
    courses.append([course[0],course[2],course[3]," ".join(course[-1]),course[5],course[6],course[4]])

# for course in courses:
#     print course

In [60]:
## in this I get all possible combinations of letters and numbers that make up netids
alpha = "abcdefghijklmnoprstuvwyz" ##I did not include q or x because none of the thousands of netids i saw had those
                                   ##letters. That is why it is in range(24) and not 26
numer = "0123456789"

three_alpha = [] ##some netids have 3 letters and 2 numbers
for i in range(24):
    for j in range(24):
        for k in range(24):
            for l in range(10):
                for m in range(10):
                    three_alpha.append(alpha[i] + alpha[j] + alpha[k] + numer[l] + numer[m])
     
two_alpha = [] ##some netids have 2 letters and 3 numbers
for i in range(24):
    for j in range(24):
        for k in range(10):
            for l in range(10):
                for m in range(10):
                    two_alpha.append(alpha[i] + alpha[j] + numer[k] + numer[l] + numer[m])
                    
three_alpha.extend(two_alpha) ##combining two types of netids
netids = three_alpha

In [145]:
start_pos = netids.index("bam76") + 1
print start_pos

58877


In [61]:
import csv ##had to use csv because if I went through all of the netids at once it would actually take over a day. I ran
           ##it overnight and it still didn't finish. I ran for a while and saved all of the a's to a CSV. There were a 
           ##lot of them and in my oppinion enough to get some preliminary data
netids2 = []
with open('Netids.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        netids2.append(row[0])
# print netids2

In [143]:
students = []
professors = []
url_section0 = "https://streamer.oit.duke.edu/ldap/people/netid/"
url_section2 = "?access_token="
print len(netids)
for netid in netids[start_pos:]:
    data = rq.get(url_section0 + netid + url_section2 + token).json()
    if data == []: ##netid does not exist
        continue
    if "emails" not in data[0].keys(): ##weird edge case
        continue
    if "duke" not in data[0]["emails"][0]: ##netid is for another university that is not duke (I found a few)
        continue
    if data[0]["primary_affiliation"] not in ["Faculty","Student"]: ##only care about teachers and students. Not staff
        continue
    name = data[0]["display_name"]
    if data[0]["primary_affiliation"] == "Student":
        students.append([netid,name])
        print netid
    if data[0]["primary_affiliation"] == "Faculty":
        if str(data[0]["department"]) == "Fin Aid & Non-Perm Positions": ##these people weren't actual teachers
            continue
        print netid
        professors.append([name,netid,data[0]["emails"][0],data[0]["phones"][0],data[0]["department"]]) ##final table
        ##I decided to add in phone number because it was really easy
print "Done!"

1958400
bae13
baf24
baf33
bag21
bag40
bag41
bah34
bah37
bak36
bak45
bal31
bal36
bal44
bal46
bam48
bam60
bam74
bam76


KeyboardInterrupt: 

In [63]:
import random
## to make up what people's majors were I wanted to be slighly realistic. I found online this website,
#"https://www.collegeraptor.com/colleges/majors/Duke-University-NC--198419". This had the major statistics for 2016 and
## i figured they were around the same. I made a list that was made up of majors weighted based on how common they were
## in 2016 below. I then assigned students majors by randomly picking one from the list
likelyhood_majors = []

with open('Major Likelyhood.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        major = row[0].split(",")[0]
        quantity = row[0].split(",")[1]
        likelyhood_majors.extend([major for x in range(int(quantity))])
top_random_number = len(likelyhood_majors)
random.shuffle(likelyhood_majors)
random.shuffle(students)
for i in range(len(students)):
    students[i].append(likelyhood_majors[random.randrange(top_random_number)])
    if i < (len(students)/3): ##I ended up giving some double majors. This will be less than 1/3 because i eventually
                              ##delete double majors where the second was the same as the first
        students[i].append(likelyhood_majors[random.randrange(top_random_number)])
    else:
        students[i].append("")
        
random.shuffle(students)

In [64]:
grad_years = [2019,2020,2021,2022]
for student in students: ##I said that around 1/4 were in each grade but not perfect
    student.append(grad_years[random.randrange(4)])
# for student in students:
#     print student

In [65]:
for student in students: ##I gave people GPAs that ranged from 2.5/4. The distribution is uniform but I know in real
                         ##life it isn't
    student.append(float(random.randrange(250,400))/100)
# for student in students:
#     print student

In [66]:
# #https://trinity.duke.edu/directory/people/a
# cleaner = []
# with open('Teachers Raw.csv', 'rb') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     for line in spamreader:
#         stringline = " ".join(line)
#         if stringline != "":
#             cleaner.append(stringline)
# # print cleaner
# cleaner.extend(["filler","filler","Contact"])
# professors = []
# name = ""
# phone = ""
# email = ""
# departments = ""
# i = 0
# while i < len(cleaner) - 3:
#     if cleaner[i+2] == "Contact" and cleaner[i] != "filler":
#         name = cleaner[i][1:-1]
# #         print name
# #     if cleaner[i+2] == "Departments":
# #         phone = cleaner[i]
# # #         print phone
#     if cleaner[i+1] == "Departments":
#         if "(" not in cleaner[i]:
#             email = cleaner[i]
#         else:
#             email = ""
        
# #         print email
#     if cleaner[i] == "Departments":
#         j = 1
#         departments = []
#         while cleaner[i + j + 2] != "Contact":
# #             print cleaner[i + j + 2]
#             if not cleaner[i+j][0].isalpha():
#                 departments.append(cleaner[i+j][1:-1])
#             else:
#                 departments.append(cleaner[i+j])
#             j = j + 1
#         professors.append([name,email, ":".join(departments)])
#     i = i + 1
    
# print professors[random.randrange(2200)][0]

In [67]:
for student in students: ##I decided that a student's favorite professor could be anybody not just people they've had as
                         ##a teacher
    student.append(professors[random.randrange(len(professors))][0])
    if student[2] == student[3]:
        student[3] = "" ##getting rid of redundant double majors

In [68]:
course_dict = {} ##keys are the department, values are the courses within the department. This is useful later
for course in courses:
#     print course
    
    if course[1] in course_dict:
        course_dict[course[1]].append(course)
    else:
        course_dict[course[1]] = []
        course_dict[course[1]].append(course)
#         print course_dict[course[1]]
# print course_dict["AAAS"]

In [69]:
def classes_taken_per_student(): ##assigning classes to students. I know its not by any means perfect, but it is pretty
                                 ##good for what it is and it does not have to be perfect
    all_classes_taken = []
    for student in students:
        classes_taken = []
        num_classes = 8*(2023 - student[-3]) - 4 ##assume that everyone takes 4 classes a semester
        major1 = student[2]
        major2 = student[3]
        
        num_classes_in_major = 0
        if major1 in course_dict.keys():
            num_classes_in_major = len(course_dict[major1])
#         print num_classes_in_major
        major_classes = min(num_classes/3,num_classes_in_major) ##major accounts for 1/3 of classes or as many that
        if major1 in course_dict.keys():                        ##exist in that department. Minimum.
            i = 0
            random.shuffle(course_dict[major1])
            while (len(classes_taken) < major_classes) and (i < len(course_dict[major1])):
                class_to_add = course_dict[major1][i]
#             print class_to_add[0]
                class_id = class_to_add[0]
                all_current_classid = [x[1] for x in classes_taken]
#                 print str(class_id), all_current_classid
                if str(class_id) not in all_current_classid: ##don't add the same class twice
                    classes_taken.append([student[0],class_to_add[0],class_to_add[-1],random.randrange(75,100)])
#                     print classes_taken
                i = i + 1
#                 print i

        num_classes_in_major = 0
        if major2 != "" and major2 in course_dict.keys(): ##repeat same process but this time for major2
            num_classes_in_major = len(course_dict[major2])
#         print num_classes_in_major
        major_classes = min(num_classes/3,num_classes_in_major)
        if major2 in course_dict.keys():
            i = 0
            random.shuffle(course_dict[major2])
            while (len(classes_taken) < major_classes) and (i < len(course_dict[major2])):
                class_to_add = course_dict[major2][i]
#             print class_to_add[0]
                class_id = class_to_add[0]
                all_current_classid = [x[1] for x in classes_taken]
#                 print str(class_id), all_current_classid
                if str(class_id) not in all_current_classid:
                    classes_taken.append([student[0],class_to_add[0],class_to_add[-1],random.randrange(75,100)])
#                     print classes_taken
                i = i + 1
#                 print i  

        classes_left = num_classes - len(classes_taken) ##number of classes not from major
#         print classes_left
        num_extra = 0
        while num_extra < classes_left:
            class_to_add = courses[random.randrange(6,len(courses))]
            class_id = class_to_add[0]
            all_current_classid = [x[1] for x in classes_taken]
            if str(class_id) not in all_current_classid:
                ##add random classes you don't already have until you get all of your classes you need
                classes_taken.append([student[0],class_to_add[0],class_to_add[-1],random.randrange(75,100)])
                num_extra = num_extra + 1

        all_classes_taken.append(classes_taken)
    
    return all_classes_taken

takes_course = classes_taken_per_student()


In [70]:
# for personsclasses in takes_course:
#     print personsclasses
#     print

In [71]:
for student in students: ##gets favorite class. I assumed here that students only have one and have to had taken the 
    for personsclasses in takes_course: ##class
        if personsclasses[0][0] == student[0]:
            student.append(personsclasses[0][1])
            break
#     print student

In [72]:
all_takes_classes = []
for personsclasses in takes_course:
    all_takes_classes.extend(personsclasses) ##change format to make it useful for later

In [73]:
random.shuffle(all_takes_classes)
course_ratings = [] ##final table
for i in range(len(all_takes_classes)/20): ##assume that 1/20 of people rate the class
    netid = all_takes_classes[i*20][0]
    course = all_takes_classes[i*20][1]
    time_frame = all_takes_classes[i*20][-2]
    quality_of_course = random.randrange(100)/10.0 ##I did random ratings in the 4 categories. Could change if needed
    quality_of_instruction = random.randrange(100)/10.0
    difficulty = random.randrange(100)/10.0
    workload = random.randrange(100)/10.0
    course_ratings.append([netid,course,time_frame,quality_of_course,quality_of_instruction,difficulty,workload])

# for rating in course_ratings:
#     print rating

In [74]:
tutors_for = [] ##final table
for i in range(len(all_takes_classes)):
    if all_takes_classes[i][-1] > 98: ##assume only people with 99 in class can tutor
        stopper = random.randrange(10) ##1/10 of these people actually decide to tutor
        if stopper == 5:
            tutors_for.append([all_takes_classes[i][0],all_takes_classes[i][1]])
# for person in tutors_for:
#     print person

In [131]:
tutor_net_ids = [x[0] for x in tutors_for]
possible_days_availible = ["M","T","W","TH","F"] ##people can work any of the days. I did a weird method to pick specifics but it works
tutor = [] ##final table
for netid in set(tutor_net_ids):
#     print netid
    pay = random.randrange(10,20) ##people get payed anywhere between 10 and 20 an hour
    days_availible = []
    for i in range(5):
        if random.randrange(2) == 1:
            days_availible.append(possible_days_availible[i])
#     print days_availible
    if len(days_availible) == 0:
        days_availible = possible_days_availible[random.randrange(5)]
    days_availible = ":".join(days_availible)
#     print days_availible
    tutor.append([netid,pay,days_availible])
# print tutor

In [98]:
tas_for = [] ##final table

terms_offered = ["2012 Fall Term", "2013 Spring Term","2013 Fall Term","2014 Spring Term","2014 Fall Term","2015 Spring Term","2015 Fall Term","2016 Spring Term","2016 Fall Term","2017 Spring Term","2017 Fall Term","2018 Spring Term","2018 Fall Term","2019 Spring Term"]

for i in range(len(all_takes_classes)):
    if all_takes_classes[i][-1] > 98: ##only people with 99 ta
        stopper = random.randrange(20) 
        if stopper == 10: ## 1/20 people TA with 99s
            netid = all_takes_classes[i][0]
            course_number = all_takes_classes[i][1]
            class_time_frame = all_takes_classes[i][-2]
            semester_ta = ""
            if class_time_frame != "2019 Spring Term": ##said that they TA the semester after they took the class and don't 
#                 print terms_offered[terms_offered.index(class_time_frame) + 1]
                
                semester_ta = terms_offered[terms_offered.index(class_time_frame) + 1] ##TA if are in spring 2019
                tas_for.append([netid,course_number,semester_ta])
# for person in tas_for:
#     print person

In [102]:
dict_courses = {}
for course in all_takes_classes: ##keys = course and term, values = people in class
    unique_course = ":".join(course[1:3])
    if unique_course not in dict_courses.keys():
        dict_courses[unique_course] = [course[0]]
    else:
        dict_courses[unique_course].append(course[0])

study_groups = [] ##final table
in_study_group = [] ##final table
study_group_id = 0

all_unique_courses = dict_courses.keys()
random.shuffle(all_unique_courses)
for i in range(len(all_unique_courses)/2): ##1/2 of classes have study groups
    specific_course = all_unique_courses[i]
    amount_of_groups = len(dict_courses[specific_course])/10 ## average of 1 study group for 10 people
    for j in range(amount_of_groups):
        size_of_group = random.randrange(3,8) ##groups range from 3-8 people
        study_groups.append([study_group_id,specific_course.split(":")[0],specific_course.split(":")[1]])
        random.shuffle(dict_courses[specific_course])
        for k in range(size_of_group):
            in_study_group.append([study_group_id,dict_courses[specific_course][k]])
        study_group_id += 1

# for group in in_study_group:
#     print group
    
# for group in study_groups:
#     print group

In [109]:
final_student = []
final_student.append(["netid","name","major1","major2","grad_year","GPA","fav_teacher","fav_class"])
for student in students:
    netid = student[0]
    name = student[1].encode('utf-8')
    major1 = student[2]
    major2 = student[3]
    grad_year = student[4]
    GPA = student[5]
    fav_teacher = student[6].encode('utf-8').replace("|","").replace(","," ")
    fav_class = student[7]
    final_student.append([netid,name,major1,major2,grad_year,GPA,fav_teacher,fav_class])

with open('student.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_student:
        spamwriter.writerow(row)

In [113]:
final_course = []
final_course.append(["number","department","level","tags","prereqs","meeting_times"])
for course in courses:
    number = course[0]
    department = course[1]
    level = course[2].strip()
    tags = course[3]
    prereqs = course[4].replace("|","").replace(","," ")
    meeting_times = course[5]
    final_course.append([number,department,level,tags,prereqs,meeting_times])

with open('course.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_course:
        spamwriter.writerow(row)

In [114]:
final_takes_course = []
final_takes_course.append(["netid","number","term","grade"])

for action in all_takes_classes:
    netid = action[0]
    number = action[1]
    term = action[2]
    grade = action[3]
    final_takes_course.append([netid,number,term,grade])

with open('takes_course.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_takes_course:
        spamwriter.writerow(row)

In [705]:
final_rates_course = []
final_rates_course.append(["netid","number","term","quality_course","quality_instructor","difficulty","workload"])

for rating in course_ratings:
    netid = rating[0]
    number = rating[1]
    term = rating[2]
    quality_course = rating[3]
    quality_instructor = rating[4]
    difficulty = rating[5]
    workload = rating[6]
    final_rates_course.append([netid,number,term,quality_course,quality_instructor,difficulty,workload])

with open('rates_course.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_rates_course:
        spamwriter.writerow(row)

In [117]:
final_professor = []
final_professor.append(["name","netid","email","phone","department"])

for professor in professors:
    name = professor[0].encode('utf-8').replace("|","").replace(","," ")
    netid = professor[1]
    email = professor[2]
    phone = professor[3].encode('utf-8')
    department = professor[4].encode('utf-8').replace("|","").replace(","," ")
    final_professor.append([name,netid,email,phone,department])

with open('professor.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_professor:
        spamwriter.writerow(row)

In [118]:
final_teaches_course = []
final_teaches_course.append(["number","term","professor","department"])

for course in teaches_course:
    number = course[0]
    term = course[1]
    professor = course[3].replace("\n","").replace("|","").replace(","," ")
    department = course[4].replace("|","").replace(","," ")
    final_teaches_course.append([number,term,professor,department])

with open('teaches_course.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_teaches_course:
        spamwriter.writerow(row)

In [119]:
final_study_group = []
final_study_group.append(["group_id","class_number","term"])

for group in study_groups:
    group_id = group[0]
    class_number = group[1]
    term = group[2]
    final_study_group.append([group_id,class_number,term])

with open('study_group.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_study_group:
        spamwriter.writerow(row)

In [3]:
final_in_study_group = []
final_in_study_group.append(["group_id","net_id"])


for group in in_study_group:
    group_id = group[0]
    net_id = group[1]
    final_in_study_group.append([group_id,net_id])

with open('in_study_group.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_in_study_group:
        spamwriter.writerow(row)

In [132]:
final_tutor = []
final_tutor.append(["netid","rate_per_hour","days_availible"])

for t in tutor:
    netid = t[0]
    rate_per_hour = t[1]
    days_availible = t[2].replace("|","").replace(","," ")
    final_tutor.append([netid,rate_per_hour,days_availible])
    
with open('tutor.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_tutor:
        spamwriter.writerow(row)

In [136]:
final_tutors_for = []
final_tutors_for.append(["net_id","course_number"])

for person in tutors_for:
    net_id = person[0]
    course_number = person[1]
    final_tutors_for.append([net_id,course_number])
    
with open('tutors_for.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_tutors_for:
        spamwriter.writerow(row)

In [140]:
final_tas_course = []
final_tas_course.append(["netid","number","term"])

for person in tas_for:
    netid = person[0]
    number = person[1]
    term = person[2]
    final_tas_course.append([netid,number,term])

with open('tas_course.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in final_tas_course:
        spamwriter.writerow(row)